In [1]:
using Unitful #https://painterqubits.github.io/Unitful.jl/stable/
#quantity * @u_str("unit abbreviation") 
using Symbolics #https://symbolics.juliasymbolics.org/dev/
#cite https://doi.org/10.48550/arXiv.2105.03949
using Latexify
using Test
#1 * @u_str("mA") is 1 milliamp
using CSV, DataFrames
#using Plots
using PlotlyJS
using Printf
using SymPy
using PDFIO
using Unzip
using Interpolations
using Plots
#plotlyjs()

In [2]:
data_dir = "C:\\Cross-Section-Data\\EXFOR\\"

"C:\\Cross-Section-Data\\EXFOR\\"

In [25]:
function check_line(line, start)
    if length(line) > length(start) - 1
        return (line[1:length(start)] == start)
    end
    return false
end

function read_spacing()

function_read_data()

check_line (generic function with 1 method)

In [28]:
path = data_dir * "gammas\\001_H_001.c4"
file_as_array = split(open(f->read(f, String), path), "\n")
space_specifiers = [index for index in 1:length(file_as_array) 
                    if check_line(file_as_array[index], "# Prj")]


13-element Vector{Int64}:
  26
  67
 110
 206
 302
 340
 376
 411
 455
 509
 557
 601
 649

In [50]:
line1 = "# Prj Targ M MF MT PXC  Energy  dEnergy  Data      dData   Cos/LO   dCos/LO   ELV/HL  dELV/HL I78 Refer (YY)              EntrySubP"
line2 = "#---><---->o<-><-->ooo<-------><-------><-------><-------><-------><-------><-------><-------><-><-----------------------><---><->o"
spacing_ends = [collect(out)[1] for out in findall(">", line2)]
spacing_starts = append!([1], [collect(out)[1] for out in findall("<", line2)])
length(spacing_ends) == length(spacing_starts)
o_indices = [length(out) for out in split(line2, "o") if length(out) > 0]

3-element Vector{Int64}:
  11
   7
 108

In [69]:
indices_with_gaps = [index for index in  1:length(spacing_ends)-1 if 
                    spacing_ends[index] != spacing_starts[index+1]-1]
missing_spacings = [spacing_ends[index]+1:spacing_starts[index+1]-1 for index in indices_with_gaps]
spacings = append!(missing_spacings, 
[spacing_starts[i]:spacing_ends[i] for i in 1:length(spacing_ends)])
spacings = append!(spacings, [length(line2):length(line2)])
spacing_dict = Dict([])
spacing_names = [spacing_dict[line1[spacing]] = spacing for spacing in spacings]
spacing_dict

19-element Vector{UnitRange{Int64}}:
 12:12
 20:22
 1:5
 6:11
 13:15
 16:19
 23:31
 32:40
 41:49
 50:58
 59:67
 68:76
 77:85
 86:94
 95:97
 98:122
 123:127
 128:130
 131:131